<div align="center">

# **Modelacion**

</div>

## Librerias

## Data

In [41]:
siniestros_ppd <- read.csv("../data/processed/Siniestros_ppd.csv")
siniestros_pph <- read.csv("../data/processed/Siniestros_pph.csv")
siniestros_pth <- read.csv("../data/processed/Siniestros_pth.csv")
siniestros_rc <- read.csv("../data/processed/Siniestros_rc.csv")
polizas_ppd <- read.csv("../data/processed/polizas_ppd.csv")
polizas_pph <- read.csv("../data/processed/polizas_pph.csv")
polizas_pth <- read.csv("../data/processed/polizas_pth.csv")
polizas_rc <- read.csv("../data/processed/polizas_rc.csv")

In [42]:
siniestros_pph$Año <- NULL
siniestros_pph$Mes <- NULL
siniestros_pph$Día <- NULL
siniestros_pph$Periodo <- NULL

In [43]:
siniestros_ppd$Año <- NULL
siniestros_ppd$Mes <- NULL
siniestros_ppd$Día <- NULL
siniestros_ppd$Periodo <- NULL

In [44]:
siniestros_pth$Año <- NULL
siniestros_pth$Mes <- NULL
siniestros_pth$Día <- NULL
siniestros_pth$Periodo <- NULL

In [45]:
siniestros_rc$Año <- NULL
siniestros_rc$Mes <- NULL
siniestros_rc$Día <- NULL
siniestros_rc$Periodo <- NULL

## Merge de las tablas

In [46]:
ppd <- merge(polizas_ppd, siniestros_ppd, by.x = "Fecha", by.y = "Fecha_siniestro")
pph <- merge(polizas_pph, siniestros_pph, by.x = "Fecha", by.y = "Fecha_siniestro")
pth <- merge(polizas_pth, siniestros_pth, by.x = "Fecha", by.y = "Fecha_siniestro")
rc <- merge(polizas_rc, siniestros_rc, by.x = "Fecha", by.y = "Fecha_siniestro")

In [47]:
ppd$X <- NULL
pph$X <- NULL
pth$X <- NULL
rc$X <- NULL

In [48]:
colnames(ppd)

[1] "Fecha"                            "Numero_polizas"                  
[3] "Suma_primas"                      "Exposicion"                      
[5] "Cantidad_siniestros"              "Valor_siniestro_incurrido_update"

## Modelo

### PPD

In [49]:
l_pph <- sum(pph$Cantidad_siniestros)/sum(pph$Exposicion)
l_ppd <- sum(ppd$Cantidad_siniestros)/sum(ppd$Exposicion)
l_pth <- sum(pth$Cantidad_siniestros)/sum(pth$Exposicion)
l_rc <- sum(rc$Cantidad_siniestros)/sum(rc$Exposicion)

In [50]:
l_pph
l_ppd
l_pth
l_rc

[1] 3.533649e-05

[1] 0.000682542

[1] 3.412524e-05

[1] 0.0001705156

In [51]:
E_pph <- l_pph * sum(pph$Exposicion)
E_ppd <- l_ppd * sum(ppd$Exposicion)
E_pth <- l_pth * sum(pth$Exposicion)
E_rc <- l_rc * sum(rc$Exposicion)

In [52]:
E_pph
E_ppd
E_pth
E_rc

[1] 740

[1] 16516

[1] 634

[1] 3538